In [70]:
import snowflake.connector
import pandas as pd

In [71]:
ctx = snowflake.connector.connect(
user='BHABADA',
password='Snowflake@1',
account='uo58731.us-east-2.aws'
)
query = "SELECT PR.PRODUCT_ID,O.ORDER_ID,O.CUSTOMER_ID, PR.PRODUCT_CATEGORY_NAME, OP.PAYMENT_VALUE,O.ORDER_DELIVERED_CUSTOMER_DATE,OC.CUSTOMER_UNIQUE_ID,OC.CUSTOMER_CITY FROM OLIST.PUBLIC.OLIST_PRODUCTS_DATASET AS PR JOIN OLIST.PUBLIC.OLIST_ORDER_ITEMS_DATASET AS OI ON PR.PRODUCT_ID = OI.PRODUCT_ID JOIN OLIST.PUBLIC.OLIST_ORDERS_DATASET O ON OI.ORDER_ID = O.ORDER_ID JOIN OLIST.PUBLIC.OLIST_CUSTOMERS_DATASET OC ON O.CUSTOMER_ID = OC.CUSTOMER_ID JOIN OLIST.PUBLIC.OLIST_ORDER_PAYMENTS_DATASET OP ON O.ORDER_ID = OP.ORDER_ID WHERE O.ORDER_STATUS = 'delivered'"
data = pd.read_sql(query, ctx)
data

,PRODUCT_ID,ORDER_ID,CUSTOMER_ID,PRODUCT_CATEGORY_NAME,PAYMENT_VALUE,ORDER_DELIVERED_CUSTOMER_DATE,CUSTOMER_UNIQUE_ID,CUSTOMER_CITY
0,1e9e8ef04dbcff4541ed26657ea517e5,e17e4f88e31525f7deef66779844ddce,f8a3e963a310aa58b60a5b1fed5bceb5,perfumaria,18.30,2018-04-27 16:42:17,b1a1199364a4a7fe27c4486ab63f550d,mogi-guacu
1,3aa071139cb16b67ca9e5dea641aaa2f,5236307716393b7114b53ee991f36956,03fc97548af8f58fefc768d12b546c9c,artes,265.99,2018-02-16 13:27:08,4b86049cb99e4aa774031daa9cd18f18,rio de janeiro
2,96bd76ec8810374ed1b65e291975717f,01f66e58769f84129811d43eefd187fb,e41819d1c95c12c9ce495b630eab8aee,esporte_lazer,87.62,2018-07-11 23:24:16,f63805d9c7edb84d92413af34b86a39c,sao paulo
3,cef67bcfe19066a932b7673e239eb23d,143d00a4f2dde4e0364ee1821577adb3,322162b5ca010c2b059cb5224dd818b1,bebes,121.84,2018-08-06 15:14:32,619e926d09b26efbd5180368b1ddc874,sao paulo
4,9dc1a7de274444849c219cff195d0b71,86cafb8794cb99a9b1b77fc8e48fbbbb,c11c31965ff02cc1d7132df8edfcbc22,utilidades_domesticas,46.19,2018-04-13 22:41:44,ad353b4fb0e294adc4eda48af73e68a6,sao paulo
...,...,...,...,...,...,...,...,...
115030,83808703fc0706a22e264b9d75f04a2e,59e88b7d3675e89aceaf86f372d3bc9a,e750210dde7e68dbd5194c0809e97f7e,informatica_acessorios,92.02,2018-01-15 20:39:35,4d1bbadcfd9bdc738e54599627870319,campo florido
115031,83808703fc0706a22e264b9d75f04a2e,6e4465d771f02e4fe335225de3c6c043,0f139d1b208a62a37671423e5eff5016,informatica_acessorios,159.54,2018-03-23 20:08:40,2cc893f1618b7317170ba2456cf76117,paramirim
115032,83808703fc0706a22e264b9d75f04a2e,6e4465d771f02e4fe335225de3c6c043,0f139d1b208a62a37671423e5eff5016,informatica_acessorios,159.54,2018-03-23 20:08:40,2cc893f1618b7317170ba2456cf76117,paramirim
115033,83808703fc0706a22e264b9d75f04a2e,6e4465d771f02e4fe335225de3c6c043,0f139d1b208a62a37671423e5eff5016,informatica_acessorios,159.54,2018-03-23 20:08:40,2cc893f1618b7317170ba2456cf76117,paramirim


In [72]:
data['ORDER_DELIVERED_CUSTOMER_DATE'] = data.ORDER_DELIVERED_CUSTOMER_DATE.apply(lambda x: pd.to_datetime(x, format="%Y-%m-%d"))

In [73]:
data = data.drop_duplicates()
data.count()

PRODUCT_ID                       104059
ORDER_ID                         104059
CUSTOMER_ID                      104059
PRODUCT_CATEGORY_NAME            102598
PAYMENT_VALUE                    104059
ORDER_DELIVERED_CUSTOMER_DATE    104051
CUSTOMER_UNIQUE_ID               104059
CUSTOMER_CITY                    104059
dtype: int64

In [74]:
data.isnull().sum()

PRODUCT_ID                          0
ORDER_ID                            0
CUSTOMER_ID                         0
PRODUCT_CATEGORY_NAME            1461
PAYMENT_VALUE                       0
ORDER_DELIVERED_CUSTOMER_DATE       8
CUSTOMER_UNIQUE_ID                  0
CUSTOMER_CITY                       0
dtype: int64

Recency

In [75]:
cust_max_purchase = data.groupby('CUSTOMER_ID').ORDER_DELIVERED_CUSTOMER_DATE.max().reset_index()
cust_max_purchase.columns = ['CUSTOMER_ID','MAX_PURCHASE_DATE']
cust_max_purchase

,CUSTOMER_ID,MAX_PURCHASE_DATE
0,00012a2ce6f8dcda20d059ce98491703,2017-11-28 15:41:30
1,000161a058600d5901f007fab4c27140,2017-07-25 18:57:33
2,0001fd6190edaaf884bcaf3d49edf079,2017-03-06 08:57:49
3,0002414f95344307404f0ace7a26f1d5,2017-09-13 20:06:02
4,000379cdec625522490c315e70c7a9fb,2018-04-13 20:21:08
...,...,...
96472,fffcb937e9dd47a13f05ecb8290f4d3e,2018-03-21 19:24:46
96473,fffecc9f79fd8c764f843e9951b11341,2018-04-10 17:20:49
96474,fffeda5b6d849fbd39689bb92087f431,2018-06-08 18:03:31
96475,ffff42319e9b2d713724ae527742af25,2018-06-18 18:33:05


In [76]:
cust_max_purchase['RECENCY'] = (cust_max_purchase['MAX_PURCHASE_DATE'].max() - cust_max_purchase['MAX_PURCHASE_DATE']).dt.days
cust_max_purchase

,CUSTOMER_ID,MAX_PURCHASE_DATE,RECENCY
0,00012a2ce6f8dcda20d059ce98491703,2017-11-28 15:41:30,322.0
1,000161a058600d5901f007fab4c27140,2017-07-25 18:57:33,448.0
2,0001fd6190edaaf884bcaf3d49edf079,2017-03-06 08:57:49,590.0
3,0002414f95344307404f0ace7a26f1d5,2017-09-13 20:06:02,398.0
4,000379cdec625522490c315e70c7a9fb,2018-04-13 20:21:08,186.0
...,...,...,...
96472,fffcb937e9dd47a13f05ecb8290f4d3e,2018-03-21 19:24:46,209.0
96473,fffecc9f79fd8c764f843e9951b11341,2018-04-10 17:20:49,189.0
96474,fffeda5b6d849fbd39689bb92087f431,2018-06-08 18:03:31,130.0
96475,ffff42319e9b2d713724ae527742af25,2018-06-18 18:33:05,120.0


In [77]:
data_user = cust_max_purchase.drop(['MAX_PURCHASE_DATE'],axis=1)

In [78]:
data_user.RECENCY.describe()

count    96469.000000
mean       275.350372
std        152.243213
min          0.000000
25%        154.000000
50%        256.000000
75%        386.000000
max        735.000000
Name: RECENCY, dtype: float64

In [79]:
data_user.head()

,CUSTOMER_ID,RECENCY
0,00012a2ce6f8dcda20d059ce98491703,322.0
1,000161a058600d5901f007fab4c27140,448.0
2,0001fd6190edaaf884bcaf3d49edf079,590.0
3,0002414f95344307404f0ace7a26f1d5,398.0
4,000379cdec625522490c315e70c7a9fb,186.0


Frequency

In [80]:
frequency_df = data.groupby(by=['CUSTOMER_ID'], as_index=False)['ORDER_DELIVERED_CUSTOMER_DATE'].count()
frequency_df.columns = ['CUSTOMER_ID','FREQUENCY']
frequency_df.head()

,CUSTOMER_ID,FREQUENCY
0,00012a2ce6f8dcda20d059ce98491703,1
1,000161a058600d5901f007fab4c27140,1
2,0001fd6190edaaf884bcaf3d49edf079,1
3,0002414f95344307404f0ace7a26f1d5,1
4,000379cdec625522490c315e70c7a9fb,1


Monetary

In [81]:
monetary_df = data.groupby(by='CUSTOMER_ID',as_index=False).agg({'PAYMENT_VALUE': 'sum'})
monetary_df.columns = ['CUSTOMER_ID','MONETARY']
monetary_df.head()

,CUSTOMER_ID,MONETARY
0,00012a2ce6f8dcda20d059ce98491703,114.74
1,000161a058600d5901f007fab4c27140,67.41
2,0001fd6190edaaf884bcaf3d49edf079,195.42
3,0002414f95344307404f0ace7a26f1d5,179.35
4,000379cdec625522490c315e70c7a9fb,107.01


In [82]:
#merge recency dataframe with frequency dataframe
temp_df = data_user.merge(frequency_df,on='CUSTOMER_ID')
temp_df.head()

,CUSTOMER_ID,RECENCY,FREQUENCY
0,00012a2ce6f8dcda20d059ce98491703,322.0,1
1,000161a058600d5901f007fab4c27140,448.0,1
2,0001fd6190edaaf884bcaf3d49edf079,590.0,1
3,0002414f95344307404f0ace7a26f1d5,398.0,1
4,000379cdec625522490c315e70c7a9fb,186.0,1


In [83]:
#merge with monetary dataframe to get a table with the 3 columns
rfm_df = temp_df.merge(monetary_df,on='CUSTOMER_ID')
#use CustomerID as index
rfm_df.set_index('CUSTOMER_ID',inplace=True)
#check the head
rfm_df.head()

,RECENCY,FREQUENCY,MONETARY
CUSTOMER_ID,,,
00012a2ce6f8dcda20d059ce98491703,322.0,1,114.74
000161a058600d5901f007fab4c27140,448.0,1,67.41
0001fd6190edaaf884bcaf3d49edf079,590.0,1,195.42
0002414f95344307404f0ace7a26f1d5,398.0,1,179.35
000379cdec625522490c315e70c7a9fb,186.0,1,107.01


In [84]:
customers_rank = rfm_df
# Create a new column that is the rank of the value of coverage in ascending order
customers_rank['RANK'] = customers_rank['MONETARY'].rank(ascending=0)

customers_rank.sort_values('RANK',ascending=True)

,RECENCY,FREQUENCY,MONETARY,RANK
CUSTOMER_ID,,,,
71901689c5f3e5adc27b1dd16b33f0b8,132.0,5,15978.65,1.0
1617b1357756262bfa56ab541c47bc16,364.0,1,13664.08,2.0
0d861a5e4dd6a9079d89e1330848f0ab,57.0,8,9258.24,3.0
cc803a2c412833101651d3f90ca7de24,133.0,2,8891.00,4.0
30bb84b541c96af98ba7d90b9ebf35d0,313.0,7,8697.99,5.0
...,...,...,...,...
da9d48cb8449edad3619186b3dd0c180,62.0,2,10.06,96473.0
a790343ca6f3fee08112d678b43aa7c5,48.0,2,9.59,96474.0
c5ea6b40204131fb48bdf66d273d7f15,426.0,2,8.86,96475.0


RFM Quartiles

In [85]:
quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])
quantiles

,RECENCY,FREQUENCY,MONETARY,RANK
0.25,154.0,1.0,61.96,24120.0
0.50,256.0,1.0,106.25,48238.0
0.75,386.0,1.0,181.78,72356.0


In [86]:
quantiles.to_dict()

{'RECENCY': {0.25: 154.0, 0.5: 256.0, 0.75: 386.0},
 'FREQUENCY': {0.25: 1.0, 0.5: 1.0, 0.75: 1.0},
 'MONETARY': {0.25: 61.96, 0.5: 106.25, 0.75: 181.78},
 'RANK': {0.25: 24120.0, 0.5: 48238.0, 0.75: 72356.0}}

In [87]:
# Arguments (x = value, p = recency, monetary_value, frequency, d = quartiles dict)
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [88]:
#create rfm segmentation table
rfm_segmentation = rfm_df
rfm_segmentation['R_Quartile'] = rfm_segmentation['RECENCY'].apply(RScore, args=('RECENCY',quantiles,))
rfm_segmentation['F_Quartile'] = rfm_segmentation['FREQUENCY'].apply(FMScore, args=('FREQUENCY',quantiles,))
rfm_segmentation['M_Quartile'] = rfm_segmentation['MONETARY'].apply(FMScore, args=('MONETARY',quantiles,))

In [89]:
rfm_segmentation.head()

,RECENCY,FREQUENCY,MONETARY,RANK,R_Quartile,F_Quartile,M_Quartile
CUSTOMER_ID,,,,,,,
00012a2ce6f8dcda20d059ce98491703,322.0,1,114.74,44679.5,2,1,3
000161a058600d5901f007fab4c27140,448.0,1,67.41,68465.0,1,1,2
0001fd6190edaaf884bcaf3d49edf079,590.0,1,195.42,21518.0,1,1,4
0002414f95344307404f0ace7a26f1d5,398.0,1,179.35,24535.0,1,1,3
000379cdec625522490c315e70c7a9fb,186.0,1,107.01,47807.5,3,1,3


In [90]:
rfm_segmentation['RFMScore'] = rfm_segmentation.R_Quartile.map(str) \
                            + rfm_segmentation.F_Quartile.map(str) \
                            + rfm_segmentation.M_Quartile.map(str)
rfm_segmentation.head()

,RECENCY,FREQUENCY,MONETARY,RANK,R_Quartile,F_Quartile,M_Quartile,RFMScore
CUSTOMER_ID,,,,,,,,
00012a2ce6f8dcda20d059ce98491703,322.0,1,114.74,44679.5,2,1,3,213
000161a058600d5901f007fab4c27140,448.0,1,67.41,68465.0,1,1,2,112
0001fd6190edaaf884bcaf3d49edf079,590.0,1,195.42,21518.0,1,1,4,114
0002414f95344307404f0ace7a26f1d5,398.0,1,179.35,24535.0,1,1,3,113
000379cdec625522490c315e70c7a9fb,186.0,1,107.01,47807.5,3,1,3,313


In [91]:
print("Best Customers: ",len(rfm_segmentation[rfm_segmentation['RFMScore']=='444']))
print('Loyal Customers: ',len(rfm_segmentation[rfm_segmentation['F_Quartile']==4]))
print("Big Spenders: ",len(rfm_segmentation[rfm_segmentation['M_Quartile']==4]))
print('Almost Lost: ', len(rfm_segmentation[rfm_segmentation['RFMScore']=='244']))
print('Lost Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='144']))
print('Lost Cheap Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='111']))

Best Customers:  879
Loyal Customers:  5952
Big Spenders:  24118
Almost Lost:  881
Lost Customers:  836
Lost Cheap Customers:  5982


In [92]:
rfm_segmentation=rfm_segmentation.reset_index()

In [93]:
rfm_segmentation.loc[rfm_segmentation['RFMScore']=='444','Type']= 'Top'
rfm_segmentation.loc[rfm_segmentation['R_Quartile']==1,'Type']= 'Lost'
rfm_segmentation.loc[(rfm_segmentation['R_Quartile']==2) | (rfm_segmentation['R_Quartile']==3) |(rfm_segmentation['F_Quartile']==3) | (rfm_segmentation['F_Quartile']==2) | (rfm_segmentation['M_Quartile']==3) | (rfm_segmentation['M_Quartile']==2) ,'Type']= 'Regular'

In [94]:
## Adding segments to the database
data=pd.merge(data,rfm_segmentation, how ='left', left_on='CUSTOMER_ID', right_on='CUSTOMER_ID')

In [95]:
top = data[data['Type']=='Top']
lost = data[data['Type']=='Lost']
regular = data[data['Type']=='Regular']

In [96]:
reg_lost = pd.concat([regular,lost])
reg_top_lost =  pd.concat([regular,lost,top])
reg_lost.to_csv("reg_lost.csv", index=False)
top.to_csv("top.csv", index=False)
reg_top_lost.to_csv("reg_top_lost.csv", index=False)

In [123]:
reg_top_lost.count()

PRODUCT_ID                       92585
ORDER_ID                         92585
CUSTOMER_ID                      92585
PRODUCT_CATEGORY_NAME            91178
PAYMENT_VALUE                    92585
ORDER_DELIVERED_CUSTOMER_DATE    92577
CUSTOMER_UNIQUE_ID               92585
CUSTOMER_CITY                    92585
RECENCY                          92577
FREQUENCY                        92585
MONETARY                         92585
RANK                             92585
R_Quartile                       92585
F_Quartile                       92585
M_Quartile                       92585
RFMScore                         92585
Type                             92585
dtype: int64

In [124]:
reg_lost.count()

PRODUCT_ID                       90575
ORDER_ID                         90575
CUSTOMER_ID                      90575
PRODUCT_CATEGORY_NAME            89179
PAYMENT_VALUE                    90575
ORDER_DELIVERED_CUSTOMER_DATE    90567
CUSTOMER_UNIQUE_ID               90575
CUSTOMER_CITY                    90575
RECENCY                          90567
FREQUENCY                        90575
MONETARY                         90575
RANK                             90575
R_Quartile                       90575
F_Quartile                       90575
M_Quartile                       90575
RFMScore                         90575
Type                             90575
dtype: int64

In [125]:
top.count()

PRODUCT_ID                       2010
ORDER_ID                         2010
CUSTOMER_ID                      2010
PRODUCT_CATEGORY_NAME            1999
PAYMENT_VALUE                    2010
ORDER_DELIVERED_CUSTOMER_DATE    2010
CUSTOMER_UNIQUE_ID               2010
CUSTOMER_CITY                    2010
RECENCY                          2010
FREQUENCY                        2010
MONETARY                         2010
RANK                             2010
R_Quartile                       2010
F_Quartile                       2010
M_Quartile                       2010
RFMScore                         2010
Type                             2010
dtype: int64

In [102]:
rfm_segmentation

,CUSTOMER_ID,RECENCY,FREQUENCY,MONETARY,RANK,R_Quartile,F_Quartile,M_Quartile,RFMScore,Type
0,00012a2ce6f8dcda20d059ce98491703,322.0,1,114.74,44679.5,2,1,3,213,Regular
1,000161a058600d5901f007fab4c27140,448.0,1,67.41,68465.0,1,1,2,112,Regular
2,0001fd6190edaaf884bcaf3d49edf079,590.0,1,195.42,21518.0,1,1,4,114,Lost
3,0002414f95344307404f0ace7a26f1d5,398.0,1,179.35,24535.0,1,1,3,113,Regular
4,000379cdec625522490c315e70c7a9fb,186.0,1,107.01,47807.5,3,1,3,313,Regular
...,...,...,...,...,...,...,...,...,...,...
96472,fffcb937e9dd47a13f05ecb8290f4d3e,209.0,1,91.91,54834.0,3,1,2,312,Regular
96473,fffecc9f79fd8c764f843e9951b11341,189.0,3,81.36,60242.0,3,4,2,342,Regular
96474,fffeda5b6d849fbd39689bb92087f431,130.0,1,63.13,71474.0,4,1,2,412,Regular
96475,ffff42319e9b2d713724ae527742af25,120.0,1,214.13,18621.0,4,1,4,414,NaN


In [104]:
rfm_segmentation['Type'].unique()

array(['Regular', 'Lost', nan, 'Top'], dtype=object)